In [19]:
import cv2
import numpy as np
import os
from sklearn.svm import SVC
import joblib
import gradio as gr


In [20]:
def extract_otsu_features(image_path):
  img = cv2.imread(image_path)
  img = cv2.resize(img, (128, 128))
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
  # Otsu
  _, otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  
  return otsu.flatten()


In [21]:
DATASET_DIR = "dataset/train"
LABELS = ["50", "100", "200", "500", "1000"]

X = []
y = []

for label in LABELS:
  folder = os.path.join(DATASET_DIR, label)
  for fname in os.listdir(folder):
    if fname.lower().endswith(('.jpg','.png','.jpeg')):
      path = os.path.join(folder, fname)
      feat = extract_otsu_features(path)
      X.append(feat)
      y.append(label)

X = np.array(X)
y = np.array(y)

np.save("features.npy", X)
np.save("labels.npy", y)
print("Fitur dan label berhasil disimpan.")


Fitur dan label berhasil disimpan.


In [22]:
X = np.load("features.npy")
y = np.load("labels.npy")

model = SVC(kernel='linear', probability=True)
model.fit(X, y)

joblib.dump(model, "coin_model.pkl")
print("Model saved.")


Model saved.


In [ ]:
import cv2
import numpy as np
import joblib

def crop_coin_strict(image, x, y, r):
  mask = np.zeros(image.shape[:2], dtype=np.uint8)
  cv2.circle(mask, (x, y), r, 255, -1)

  coin = cv2.bitwise_and(image, image, mask=mask)

  # crop bounding box lingkaran agar tidak ada area luar
  x1 = max(x-r, 0)
  y1 = max(y-r, 0)
  x2 = min(x+r, image.shape[1])
  y2 = min(y+r, image.shape[0])

  coin = coin[y1:y2, x1:x2]

  return coin


def detect_coins(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Otsu
  _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

  # bersihkan noise kecil
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
  morph = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=2)

  # cari kontur
  cnts, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  circles = []
  for c in cnts:
    area = cv2.contourArea(c)
    if area < 3000:  # filter noise
      continue

    (x, y), r = cv2.minEnclosingCircle(c)
    circles.append((int(x), int(y), int(r)))

  return circles

def preprocess(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (128,128))
  _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  return th.flatten().reshape(1, -1)


def detect_and_label(image):
  img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  circles = detect_coins(image)

  if len(circles) == 0:
    return img

  model = joblib.load("coin_model.pkl")

  for x, y, r in circles:
    top = max(0, y-r)
    bottom = min(img.shape[0], y+r)
    left = max(0, x-r)
    right = min(img.shape[1], x+r)

    coin = img[top:bottom, left:right]
    if coin.size == 0:
      continue

    gray = cv2.cvtColor(coin, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, (128,128))
    _, otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    feat = otsu.flatten().reshape(1,-1)

    coin_img = crop_coin_strict(image, x, y, r)
    feat = preprocess(coin_img)
    pred = model.predict(feat)[0]

    cv2.circle(img, (x, y), r, (0,255,0), 4)
    cv2.putText(img, pred, (x-r, y-r), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 3)

  return img


In [24]:
def predict_gradio(input_image):
  return detect_and_label(input_image)

demo = gr.Interface(
  fn=predict_gradio,
  inputs=gr.Image(type="numpy"),
  outputs="image",
  title="Deteksi Koin Rupiah"
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py", line 2125, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py", line 1607, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  